In [9]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.cohort.cohort import CohortProcessor
from utils.ethogram import ethogram

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort4/database.xlsx'
# fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       4     Day  2020_09_22_14_09_44_904384_compressed.mp4     P15             6                  NaN        1           1  2020_09_22_14_09_44_904384_compressed_Day.slp   14:09:44
1           1       4     Day  2020_09_22_15_16_28_547483_compressed.mp4     P15             6                  NaN        1           1  2020_09_22_15_16_28_547483_compressed_Day.slp   15:16:28
2           2       4     Day  2020_09_22_16_23_20_653413_compressed.mp4     P15             6                  NaN        1           1  2020_09_22_16_23_20_653413_compressed_Day.slp   16:23:20
3           3       4     Day  2020_09_22_17_30_03_197326_compressed.mp4     P15             6                  NaN        1           1  2020_09_22_17_30_03_197326_compressed_Day.slp   17:30:03
4           4       

In [14]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################

# NOTE The hybrid video processing seems to break when using parallel flag

#
cohort.n_cores = 30
cohort.parallel = True
cohort.interpolate_flag = True
cohort.exclude_huddles = False   # Don't use this here yet; 
                                 # NEED TO FIRST FIX THE HUDDLES THEN RUN THIS
                                 # See cell 3 below which does this after
cohort.fps = 24
cohort.preprocess_feature_tracks()

#
print ("DONE...")

Found # of Day/night slp files:  282


100%|███████████████████████████████████████| 282/282 [00:00<00:00, 9893.05it/s]

Found # of both slp files:  104



100%|███████████████████████████████████████| 104/104 [00:00<00:00, 5968.42it/s]


DONE...


In [15]:
#########################################################
############## PREPROCESS HUDDLES #######################
#########################################################

# NOTE The hybrid video processing seems to break when using parallel flag

# 
cohort.n_cores = 30 
cohort.parallel = False 
cohort.recompute_flag = False 

# track fix parms
cohort.fix_track_flag = True       
cohort.max_jump_allowed = 50   # maximum distance that a gerbil/huddle can move in 1 frame
cohort.max_dist_to_join = 50   # maximum distnace between 2 chunks that can safely be merged
cohort.min_chunk_len = 24      # shortest duration of huddle; everything else gets deleted

# interploation params
cohort.interpolate_flag = True 
cohort.max_distance_huddle = 50           # how far away we can merge huddles together (pixels)
cohort.max_time_to_join_huddle = 5*24    # how far in time can we merge huddle chunks (seconds x frames)
cohort.min_huddle_time = 24            # minimum huddle duration in frames
              
#
cohort.fps = 24
cohort.preprocess_huddle_tracks()

# 
print ("DONE...")



# of huddle day/night huddle files found:  282


100%|██████████████████████████████████████| 282/282 [00:00<00:00, 17691.40it/s]


# of huddle hybrid huddle files found:  104


100%|██████████████████████████████████████| 104/104 [00:00<00:00, 49858.00it/s]

light_switch time:  23:41 34104 ['day', 'night']


light_switch time:  23:41 34104 ['day', 'night']
light_switch time:  7:53 11352 ['night', 'day']
light_switch time:  7:53 11352 ['night', 'day']
light_switch time:  57:28 82752 ['day', 'night']
light_switch time:  57:28 82752 ['day', 'night']
light_switch time:  38:03 54792 ['night', 'day']
light_switch time:  38:03 54792 ['night', 'day']
light_switch time:  40:01 57624 ['day', 'night']
light_switch time:  40:01 57624 ['day', 'night']
light_switch time:  31:40 45600 ['night', 'day']
light_switch time:  31:40 45600 ['night', 'day']
light_switch time:  23:28 33792 ['day', 'night']
light_switch time:  23:28 33792 ['day', 'night']
light_switch time:  14:48 21312 ['night', 'day']
light_switch time:  14:48 21312 ['night', 'day']
light_switch time:  6:06 8784 ['day', 'night']
light_switch time:  6:06 8784 ['day', 'night']
light_switch time:  53:52 77568 ['day', 'night']
light_switch time:  53:52 77568 ['day', 'night']
light_switch time:  44:13 63672 ['night', 'day']
light_switch time:  44:13 

In [16]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
cohort.huddle_min_distance = 200    # min distance from huddle in pixels for feature to be kept

# this uses the _spine.npy data for tracks; and _huddle_spine_fixed_interpolated.npy for huddles
# if we want to use something else, we can also code it
cohort.parallel = True
cohort.n_cores = 30
cohort.remove_huddles_from_feature_tracks()

#
print ("DONE...")

# file pairs found:  334  (if less  than above, please check missing)


100%|█████████████████████████████████████████| 334/334 [05:05<00:00,  1.09it/s]

DONE...


In [20]:
d = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_17_30_03_197326_compressed_all_frames_with_switches.npy')
print (d.shape)

print ("first swtich frames info: ", d[0])

track = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_17_30_03_197326_compressed_Day_spine.npy')

print ("track,: ", track.shape)

frame_ids = [0,1,2]
for frame_id in frame_ids:
    print (track[d[frame_id][0],d[frame_id][2]])
    print (track[d[frame_id][1],d[frame_id][3]])

(27, 4)
first swtich frames info:  [13745 13746     2     3]
track,:  (86407, 6, 2)
[316.18600464 659.28106689]
[327.49484253 667.49035645]
[327.49484253 667.49035645]
[339.19869995 672.85693359]
[344.81295776 671.51342773]
[340.71337891 664.4163208 ]
